In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import duckdb
import pandas as pd
import sqlalchemy

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///my-db.duckdb

In [3]:
from uniprot_log_file_parser import db

In [4]:
dbc = db.get_db_connection('foo.duckdb')

In [32]:
dbc.sql("DROP TABLE useragent")
db.setup_tables(dbc, "uniprotkb")

In [5]:
df = pd.read_csv('useragent.csv')
df['id'] = df.index
df['string'] = df['string'].fillna("")
df["type"] = df["type"].astype("category")
# df["id"] = df["id"].astype("int32")

In [21]:
db.update_useragents(dbc, df2)

In [16]:
n = pd.DataFrame([{
    "string": "foo",
    "family":  "Chrome",
    "id": len(df),
}])

In [20]:
df2 = pd.concat([df, n])

In [ ]:
df.loc[len(df.index)] = 

In [ ]:
dict = {'Name':['Martha', 'Tim', 'Rob', 'Georgia'],
        'Maths':[87, 91, 97, 95],
        'Science':[83, 99, 84, 76]
       }
  
df = pd.DataFrame(dict)
  
display(df)
  
df.loc[len(df.index)] = ['Amy', 89, 93] 

In [5]:
df2 = db.get_useragents(dbc)

In [6]:
df2

,string,family,type,major,id
0,,Other,unknown,False,0
1,"""Mozilla/5.0 (Windows NT 10.0; Win64; x64) App...",Edge,browser,True,1
2,"""Mozilla/5.0 (Windows NT 10.0; Win64; x64) App...",Chrome,browser,True,2
3,"""Mozilla/5.0 (iPod; U; CPU iPhone OS 2_1 like ...",Mobile Safari,browser,True,3
4,-,Other,unknown,False,4
...,...,...,...,...,...
91831,yacybot (/global; amd64 Linux 6.0.12-gentoo; j...,yacybot,bot,False,91831
91832,yacybot (/global; amd64 Linux 6.0.7-gentoo; ja...,yacybot,bot,False,91832
91833,yacybot (/global; amd64 Linux 6.0.9-gentoo; ja...,yacybot,bot,False,91833
91834,yacybot (/global; x86_64 Mac OS X 10.14.6; jav...,yacybot,bot,False,91834


In [36]:
df2.dtypes

string      object
family      object
type      category
major         bool
id           int32
dtype: object

In [37]:
df.dtypes

string      object
family      object
type      category
major         bool
id           int64
dtype: object

In [38]:
for c in df.columns:
    if c != 'type':
        print(c, sum(df[c] != df2[c]))

string 0
family 0
major 0
id 0


In [ ]:
dbc.sql("SELECT * FROM useragent")

In [9]:
dbc.sql("DESCRIBE SELECT * FROM useragent;")

┌─────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │  column_type   │  null   │   key   │ default │  extra  │
│   varchar   │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ id          │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ string      │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ family      │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ type        │ useragent_type │ YES     │ NULL    │ NULL    │ NULL    │
│ major       │ BOOLEAN        │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴────────────────┴─────────┴─────────┴─────────┴─────────┘

In [30]:
%%sql
CREATE TYPE useragent_type AS ENUM ('browser', 'programmatic', 'bot', 'unknown');
CREATE TABLE useragent
 (
     string VARCHAR,
     family VARCHAR,
     type useragent_type,
     major BOOLEAN,
     PRIMARY KEY (string, family, type, major)
 )
INSERT INTO useragent SELECT * FROM read_csv_auto('useragent.csv');

""


In [4]:
%%time
%%sql
SELECT
  COUNT(*)
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true AND
  useragent.type = 'browser'

CPU times: user 1min 6s, sys: 4.02 s, total: 1min 10s
Wall time: 20.2 s


,count_star()
0,86373098


In [6]:
%%time
%%sql
df << SELECT
  uniprotkb.datetime,
  uniprotkb.method,
  uniprotkb.request,
  uniprotkb.status,
  uniprotkb.bytes,
  uniprotkb.referrer,
  uniprotkb.useragent,
  useragent.family,
  useragent.type
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  request ILIKE '%ftev\_%' AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true

Returning data to local variable df
CPU times: user 5min, sys: 1min 22s, total: 6min 23s
Wall time: 8min 38s


In [8]:
df

""


In [7]:
df.loc[0,'request']

KeyError: 'request'